In [1]:
import os
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
from keras.models import Model
from keras.optimizers import Adam


2023-06-08 14:49:03.288162: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 14:49:03.418295: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neo/swarm_project_ws/devel/lib:/opt/ros/noetic/lib
2023-06-08 14:49:03.418312: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-08 14:49:04.236217: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnv

In [2]:
NUM_BATCHES = 600
BATCH_SIZE = 512
PLOT_EVERY = 1
GRID_RESOLUTION = 400
LATENT_DIM = 1

In [3]:
def uniform_to_normal(z):
    '''
    Map a value from ~U(-1, 1) to ~N(0, 1)
    '''
    norm = stats.norm(0, 1)
    return norm.ppf((z+1)/2)

In [4]:
def generate_noise(samples, dimensions=2):
    '''
    Generate a matrix of random noise in [-1, 1] with shape (samples, dimensions) 
    '''
    return np.random.uniform(-1, 1, (samples, dimensions))


In [5]:
def build_generator(LATENT_DIM, output_dim):
    '''
    Build a generator mapping (R, R) to ([-1,1], [-1,1])
    '''
    input_layer = layers.Input((LATENT_DIM,))
    X = input_layer
    for i in range(4):
        X = layers.Dense(16)(X)
        X = layers.LeakyReLU(0.1)(X)
    output_layer = layers.Dense(output_dim)(X)
    G = Model(input_layer, output_layer)
    return G

In [6]:
def build_discriminator(dim):
    '''
    Build a discriminator mapping (R, R) to [0, 1]
    '''
    input_layer = layers.Input((dim,))
    X = input_layer
    for i in range(2):
        X = layers.Dense(64)(X)
        X = layers.LeakyReLU(0.1)(X)
    output_layer = layers.Dense(1, activation='sigmoid')(X)
    D = Model(input_layer, output_layer)
    D.compile(Adam(learning_rate=0.002, beta_1=0.5),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    return D

In [7]:
def build_GAN(G, D, LATENT_DIM):
    '''
    Given a generator and a discriminator, build a GAN
    '''
    D.trainable = False
    input_layer = layers.Input((LATENT_DIM,))
    X = G(input_layer)
    output_layer = D(X)
    GAN = Model(input_layer, output_layer)
    GAN.compile(Adam(learning_rate=0.001, beta_1=0.5),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    return GAN


In [8]:
### Visualizations
grid_latent = np.linspace(-1, 1, 203)[1:-1].reshape((-1, 1))
grid_sample = np.linspace(-3, 3, 603)[1:-1].reshape((-1, 1))
test_noise = generate_noise(5000, LATENT_DIM)
test_samples = uniform_to_normal(test_noise)
def plot(G, D, step, step_count, D_accuracy, D_loss, G_accuracy, G_loss, filename):
    '''
    Plots for the GAN gif
    '''
    f, ax = plt.subplots(2, 2, figsize=(8,8))
    f.suptitle(f'\n       {step:05d}', fontsize=10)


    # [0, 0]: plot loss and accuracy
    ax[0, 0].plot(step_count, 
                    G_loss,
                    label='G loss',
                    c='darkred',
                    zorder=50,
                    alpha=0.8,)
    ax[0, 0].plot(step_count, 
                    G_accuracy,
                    label='G accuracy',
                    c='lightcoral',
                    zorder=40,
                    alpha=0.8,)
    ax[0, 0].plot(step_count, 
                    D_loss,
                    label='D loss',
                    c='darkblue',
                    zorder=55,
                    alpha=0.8,)
    ax[0, 0].plot(step_count, 
                    D_accuracy,
                    label='D accuracy',
                    c='cornflowerblue',
                    zorder=45,
                    alpha=0.8,)
    ax[0, 0].set_xlim(-5, max(max(step_count), NUM_BATCHES)+5)
    ax[0, 0].set_ylim(-0.05, 1.55)
    ax[0, 0].set_xlabel('Epoch')
    ax[0, 0].legend(loc=1)

    # [0, 1]: Plot actual samples and fake samples

    fake_samples = G.predict(test_noise, batch_size=len(test_noise))
    #sns.kdeplot(test_samples.flatten(), c='blue', alpha=0.6, label='Real', ax=ax[0, 1], shade=True)
    x_vals = np.linspace(-3, 3, 301)
    y_vals = stats.norm(0,1).pdf(x_vals)
    ax[0, 1].plot(x_vals, y_vals, color='blue', label='real')
    ax[0, 1].fill_between(x_vals, np.zeros(len(x_vals)), y_vals, color='blue', alpha=0.6)
    #sns.kdeplot(test_samples.flatten(), c='blue', alpha=0.6, label='Real', ax=ax[0, 1], shade=True)
    sns.kdeplot(fake_samples.flatten(), color='red', alpha=0.6, label='GAN', ax=ax[0, 1], shade=True)
    ax[0, 1].set_xlim(-3, 3)
    ax[0, 1].set_ylim(0, 0.82)
    ax[0, 1].legend(loc=1)
    ax[0, 1].set_xlabel('Sample Space')
    ax[0, 1].set_ylabel('Probability Density')


    # [1, 0]: Confident real input
    grid_latent = np.linspace(-1, 1, 103)[1:-1].reshape((-1, 1))
    true_mappings = uniform_to_normal(grid_latent)
    GAN_mapping = G.predict(grid_latent)
    ax[1, 0].scatter(grid_latent.flatten(), true_mappings.flatten(), 
                edgecolor='blue', facecolor='None', s=5, alpha=1, 
                linewidth=1, label='Real Mapping')
    ax[1, 0].scatter(grid_latent.flatten(), GAN_mapping.flatten(), 
                edgecolor='red', facecolor='None', s=5, alpha=1, 
                linewidth=1, label='GAN Mapping')
    ax[1, 0].legend(loc=8)
    ax[1, 0].set_xlim(-1, 1)
    ax[1, 0].set_ylim(-3, 3)
    ax[1, 0].set_xlabel('Latent Space')
    ax[1, 0].set_ylabel('Sample Space')

    # [1, 1]: Confident real ouput
    confidences = D.predict(grid_sample, batch_size=BATCH_SIZE).flatten()
    ax[1, 1].plot(grid_sample.flatten(), confidences, c='k')
    lower, upper = -3, 3
    for i in range(0, len(confidences), 50):
        if i ==0:
            continue
        ax[1, 1].plot([i / len(confidences) * (upper - lower) + lower, ]*2,
                        [0, confidences[i]], c='k')
    ax[1, 1].plot([lower, lower, upper, upper], [confidences[0], 0, 0, confidences[-1]], c='k')
    ax[1, 1].fill_between(grid_sample.flatten(), np.zeros(len(confidences)), confidences, color='k', alpha=0.6)
    ax[1, 1].set_xlabel('Sample Space Value')
    ax[1, 1].set_ylabel('Discriminator Confidence')
    ax[1, 1].set_xlim(lower, upper)
    ax[1, 1].set_ylim(-0.00, 1.00)

    plt.tight_layout()
    plt.savefig(filename)
    plt.close()
    return

In [9]:
## Set up directories
paths = ['ims', 
            'ims/1_1D_normal', 
                'ims/1_1D_normal/a', 
                'ims/1_1D_normal/b', 
                'ims/1_1D_normal/c', 
        ]
for i in paths:
    if not os.path.exists(i):
        os.makedirs(i)


In [10]:
## Part 1b: 1D uniform to 1D normal GAN
### Setup

test_noise = generate_noise(1000, LATENT_DIM)
test_samples = uniform_to_normal(test_noise)
G = build_generator(LATENT_DIM, 1)
D = build_discriminator(1)
GAN = build_GAN(G, D, LATENT_DIM)

step_count = []
D_accuracy = []
G_accuracy = []
D_loss = []
G_loss = []
count = 0

2023-06-08 14:51:03.984656: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-08 14:51:03.985138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neo/swarm_project_ws/devel/lib:/opt/ros/noetic/lib
2023-06-08 14:51:03.985225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neo/swarm_project_ws/devel/lib:/opt/ros/noetic/lib
2023-06-08 14:51:03.985276: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library

In [11]:
for step in range(NUM_BATCHES):
    print(f'1b: {step}/{NUM_BATCHES}', end='\r')
    # Train discriminator
    D.trainable = True
    real_data = uniform_to_normal(generate_noise(BATCH_SIZE // 2, LATENT_DIM))
    fake_data = G.predict(generate_noise(BATCH_SIZE // 2, LATENT_DIM), batch_size=BATCH_SIZE // 2)
    data = np.concatenate((real_data, fake_data), axis=0)
    real_labels = np.ones((BATCH_SIZE // 2, 1))
    fake_labels = np.zeros((BATCH_SIZE // 2, 1))
    labels = np.concatenate((real_labels, fake_labels), axis=0)
    _D_loss, _D_accuracy = D.train_on_batch(data, labels)

    # Train generator
    D.trainable = False
    noise = generate_noise(BATCH_SIZE, LATENT_DIM)
    labels = np.ones((BATCH_SIZE, 1))
    _G_loss, _G_accuracy = GAN.train_on_batch(noise, labels)
    
    if step % PLOT_EVERY == 0:
        step_count.append(step)
        D_loss.append(_D_loss)
        D_accuracy.append(_D_accuracy)
        G_loss.append(_G_loss)
        G_accuracy.append(_G_accuracy)
        plot(G=G, 
             D=D,
             step=step+1,
             step_count=step_count,
             D_accuracy=D_accuracy,
             D_loss=D_loss,
             G_accuracy=G_accuracy,
             G_loss=G_loss,
             filename=f'ims/1_1D_normal/b/1b.{count:03d}.png')
        count += 1
print()
os.system(f'ffmpeg -r 20 -i ims/1_1D_normal/b/1b.%03d.png'
              f' -crf 15 ims/1_1D_normal/b/1b.mp4')

2/2 [==============================] - 0s 2ms/step


/tmp/ipykernel_2220369/4251343495.py:53: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(fake_samples.flatten(), color='red', alpha=0.6, label='GAN', ax=ax[0, 1], shade=True)


2/2 [==============================] - 0s 1ms/step


/tmp/ipykernel_2220369/4251343495.py:53: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(fake_samples.flatten(), color='red', alpha=0.6, label='GAN', ax=ax[0, 1], shade=True)


2/2 [==============================] - 0s 1ms/step


...

/tmp/ipykernel_2220369/4251343495.py:53: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(fake_samples.flatten(), color='red', alpha=0.6, label='GAN', ax=ax[0, 1], shade=True)


2/2 [==============================] - 0s 2ms/step


/tmp/ipykernel_2220369/4251343495.py:53: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(fake_samples.flatten(), color='red', alpha=0.6, label='GAN', ax=ax[0, 1], shade=True)


sh: 1: ffmpeg: not found


32512

In [ ]:
total_steps = 30000
for step in range(NUM_BATCHES, total_steps):
    print(f'1c: {step}/{total_steps}', end='\r')
    # Train discriminator
    D.trainable = True
    real_data = uniform_to_normal(generate_noise(BATCH_SIZE // 2, LATENT_DIM))
    fake_data = G.predict(generate_noise(BATCH_SIZE // 2, LATENT_DIM), batch_size=BATCH_SIZE // 2)
    data = np.concatenate((real_data, fake_data), axis=0)
    real_labels = np.ones((BATCH_SIZE // 2, 1))
    fake_labels = np.zeros((BATCH_SIZE // 2, 1))
    labels = np.concatenate((real_labels, fake_labels), axis=0)
    _D_loss, _D_accuracy = D.train_on_batch(data, labels)

    # Train generator
    D.trainable = False
    noise = generate_noise(BATCH_SIZE, LATENT_DIM)
    labels = np.ones((BATCH_SIZE, 1))
    _G_loss, _G_accuracy = GAN.train_on_batch(noise, labels)

    if step % PLOT_EVERY == 0:
        step_count.append(step)
        D_loss.append(_D_loss)
        D_accuracy.append(_D_accuracy)
        G_loss.append(_G_loss)
        G_accuracy.append(_G_accuracy)

1/1 [==============================] - 0s 20ms/step


2023-06-08 15:03:34.715618: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


1/1 [==============================] - 0s 19ms/step


2023-06-08 15:11:13.870928: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


1/1 [==============================] - 0s 16ms/step


In [ ]:
plot(G=G, 
     D=D,
     step=step+1,
     step_count=step_count,
     D_accuracy=D_accuracy,
     D_loss=D_loss,
     G_accuracy=G_accuracy,
     G_loss=G_loss,
     filename=f'ims/1_1D_normal/c/1c.{total_steps:03d}.png')
G.save("ims/1_1D_normal/c/G.h5")
D.save("ims/1_1D_normal/c/D.h5")


2/2 [==============================] - 0s 3ms/step


/tmp/ipykernel_2220369/4251343495.py:53: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(fake_samples.flatten(), color='red', alpha=0.6, label='GAN', ax=ax[0, 1], shade=True)


In [16]:

print(fake_data)

[[ 0.6646419 ]
 [ 0.38621628]
 [-1.6514962 ]
 [ 0.3929608 ]
 [-1.2866645 ]
 [-1.2013404 ]
 [-0.04338103]
 [ 0.40493065]
 [ 2.3859143 ]
 [ 2.3750048 ]
 [-0.56560683]
 [-0.12308069]
 [-1.6559303 ]
 [-1.9589312 ]
 [-2.1278217 ]
 [ 1.8273299 ]
 [ 0.4557156 ]
 [ 0.5699964 ]
 [ 1.078537  ]
 [ 0.38043302]
 [-2.2705092 ]
 [-1.9439101 ]
 [ 0.946715  ]
 [ 0.30496094]
 [-1.0770917 ]
 [ 0.491966  ]
 [-1.2224164 ]
 [-0.23314917]
 [ 0.75849384]
 [ 0.25172567]
 [-0.2707524 ]
 [-0.13205937]
 [ 0.41053498]
 [-0.479999  ]
 [ 0.35611337]
 [-1.8672667 ]
 [ 0.27391735]
 [ 2.1370134 ]
 [ 0.53803796]
 [-2.1742048 ]
 [-1.1453884 ]
 [-0.9259301 ]
 [ 1.432311  ]
 [-0.05354583]
 [ 0.4844699 ]
 [ 2.2699425 ]
 [-1.7236795 ]
 [ 0.48200816]
 [ 0.685404  ]
 [ 0.04479387]
 [ 0.31398958]
 [ 1.6006122 ]
 [ 0.38921365]
 [-2.1976151 ]
 [ 0.45270336]
 [-0.07926093]
 [ 2.2947314 ]
 [ 0.39542925]
 [ 0.98607063]
 [ 0.38668242]
 [ 1.0932138 ]
 [-0.52024174]
 [-0.3105913 ]
 [ 0.38622773]
 [ 0.80503184]
 [ 0.46621266]
 [ 1.36187